## Exploring the CONLL 2003 dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-'

#### Install the autolabel library

In [2]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [2]:
from autolabel import get_data

get_data('tweebank')

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [10]:
import json

from autolabel import LabelingAgent

In [11]:
# load the config
with open('config_tweebank.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `named_entity_recognition` (since it's a named entity recognition task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at extracting Person, Organization, Location, and Miscellaneous entities...` (how we describe the task to the LLM)
* `prompt.labels`: `[
            "Location",
            "Organization",
            "Person",
            "Miscellaneous"
        ]` (the full list of labels to choose from)
* `prompt.few_shot_num`: 3 (how many labeled examples to provide to the LLM)

In [12]:
config

{'task_name': 'PersonLocationOrgMiscNER',
 'task_type': 'named_entity_recognition',
 'dataset': {'label_column': 'CategorizedLabels',
  'text_column': 'example',
  'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at extracting Person, Organization, Location, and Miscellaneous entities from text. Your job is to extract named entities mentioned in text, and classify them into one of the following categories.\nCategories:\n{labels}\n ',
  'labels': ['Location', 'Organization', 'Person', 'Miscellaneous'],
  'example_template': 'Example: {example}\nOutput:\n{CategorizedLabels}',
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 10}}

In [13]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [14]:
# dry-run -- this tells us how much this will cost and shows an example prompt
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $2.5739 │
│ Number of Examples       │ 1000    │
│ Average cost per example │ $0.0026 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at extracting Person, Organization, Location, and Miscellaneous entities from text. Your job is 
to extract named entities mentioned in text, and classify them into one of the following categories.
Categories:
Location
Organization
Person
Miscellaneous
 

You will return the answer in CSV format, with two columns seperated by the % character. First column is the 
extracted entity and second column is the category. Rows in the CSV are separated by new line character.

Some examples with their output answers are provided below:

Example: RT @USER1316 : This is so cute 😍 URL1099
Output:


Example: RT @USER1796 : Today be your best ! #mondaymotivation @USER1083 @USER2417 @USER1974 @USER791 @USER2146 
@USER605 URL221 …
Output:


Example: @USER516 follow me back yo !
Output:


Example: RT @USER294 : Make money just signing up to SmashFund .... harness the power of your social media contacts
URL790 htt …
Output:
SmashFund%Organization

Example: 2015 - 2016 Donald Publishing Award winners K-5 . #AuthorsTea @USER373 @USER1834 URL949
Output:
2015 - 2016 Donald Publishing Award%Miscellaneous

Example: RT @USER1343 : How high are you ? URL1237
Output:


Example: RT @USER1537 : I want it to be warm ☹ I have cute dresses that I wan na wear !!
Output:


Example: RT @USER340 : GET OUT OF MY SWAMP URL983
Output:


Example: @USER2471 STHERIOUTHLY .
Output:


Example: RT @USER2254 : Best Of Lover 's : Creat Free Website form the Best Free Hosting At All ... URL816
Output:


Now I want you to label the following example:
Example: RT @USER2660 : I want to win @USER1865 's books giveaway ! #esthetichadidgiveaways P.S. Saved accounts are
allowed ! 💗🌸🌺 URL885 …
Output:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [15]:
# now, do the actual labeling
ds = agent.run(ds, max_items=100)

Output()

2023-10-04 20:43:47 openai INFO: error_code=502 error_message='Bad gateway.' error_param=None error_type=cf_bad_gateway message='OpenAI API error received' stream_error=False
2023-10-04 20:43:47 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 04 Oct 2023 20:43:47 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '811033afd9cfe807-DFW', 'alt-svc': 'h3=":443"; ma=864

Actual Cost: 0.0605

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ support ┃ completion_rate ┃ f1_exact ┃ f1_strict ┃ f1_partial ┃ f1_ent_type ┃ accuracy ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 100     │ 1.0             │ 0.3063   │ 0.2937    │ 0.3219     │ 0.3187      │ 0.746    │
└─────────┴─────────────────┴──────────┴───────────┴────────────┴─────────────┴──────────┘